In [2]:
#link google colab with google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#first to import libraries
from keras.layers import Input, Lambda, Dense, Flatten, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings   
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
train_data_dir = '/content/drive/MyDrive/Data_Small2/train'
val_data_dir = '/content/drive/MyDrive/Data_Small2/val'
test_data_dir = '/content/drive/MyDrive/Data_Small2/test'
img_width, img_height = 128, 128
batch_size = 32

In [5]:
# Preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

Found 2014 images belonging to 2 classes.
Found 507 images belonging to 2 classes.
Found 514 images belonging to 2 classes.


In [7]:
# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [8]:
# Add new layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [9]:
# Create a new model
modelA = Model(inputs=base_model.input, outputs=predictions)

In [10]:
# Freeze layers
for layer in base_model.layers:
    layer.trainable = False 

In [11]:
# Compile the model
modelA.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [14]:
# Train the model
modelA.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=150,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size)

Epoch 1/150


<ipython-input-14-3446c55fad91>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  modelA.fit_generator(


62/62 [==============================] - 10s 168ms/step - loss: 0.6632 - accuracy: 0.5979 - val_loss: 0.6619 - val_accuracy: 0.5896
Epoch 2/150
62/62 [==============================] - 10s 165ms/step - loss: 0.6591 - accuracy: 0.6004 - val_loss: 0.6596 - val_accuracy: 0.5875
Epoch 3/150
62/62 [==============================] - 10s 162ms/step - loss: 0.6559 - accuracy: 0.5999 - val_loss: 0.6565 - val_accuracy: 0.5896
Epoch 4/150
62/62 [==============================] - 10s 164ms/step - loss: 0.6539 - accuracy: 0.5989 - val_loss: 0.6516 - val_accuracy: 0.5979
Epoch 5/150
62/62 [==============================] - 11s 170ms/step - loss: 0.6516 - accuracy: 0.5999 - val_loss: 0.6463 - val_accuracy: 0.5979
Epoch 6/150
62/62 [==============================] - 10s 164ms/step - loss: 0.6486 - accuracy: 0.5994 - val_loss: 0.6456 - val_accuracy: 0.5938
Epoch 7/150
62/62 [==============================] - 10s 168ms/step - loss: 0.6456 - accuracy: 0.6029 - val_loss: 0.6419 - val_accuracy: 0.5979
Epoc

In [15]:
import pickle

# assume that `model` is a trained machine learning model
with open('model.pkl', 'wb') as f:
    pickle.dump(modelA, f)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......conv2d_10
.........vars
............0
............1
......conv2d_11
.........vars
............0
............1
......conv2d_12
.........vars
............0
............1
......conv2d_2
.........vars
............0
............1
......conv2d_3
.........vars
............0
............1
......conv2d_4
.........vars
............0
............1
......conv2d_5
.........vars
............0
............1
......conv2d_6
.........vars
............0
............1
......conv2d_7
.........vars
............0
............1
......conv2d_8
.........vars
............0
............1
......conv2d_9
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......global_average_pooling2d
.........vars
......input_layer
.........

In [17]:
modelA.save('/content/drive/MyDrive/models/my_model')